In [225]:
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split


In [226]:
df_train = pd.read_csv('Cleaned_train.csv', index_col=["ID"])
df_test = pd.read_csv('Cleaned_test.csv', index_col=["ID"])
test_id = df_test.index
common_cols = list(set(df_train.columns) & set(df_test.columns))
df_train = df_train[common_cols]
df_test = df_test[common_cols]

In [227]:
df_test.columns.tolist()

['Automatic',
 'Alloy wheels',
 'city_cars',
 'Rok_produkcji',
 'Panoramic roof',
 'On-board computer',
 'Central locking',
 'Rear wheels',
 'Xenon lights',
 'Kraj_pochodzenia',
 'Sunroof',
 'Active cruise control',
 'Rear side airbags',
 'CD changer',
 'Shift paddles',
 'Auxiliary heating',
 'ASR (traction control)',
 'convertible',
 'Cena',
 'Heated windscreen',
 'Gasoline + CNG',
 'Aftermarket radio',
 'Electrochromic side mirrors',
 'Heated side mirrors',
 'Front parking sensors',
 'Hybrid',
 'Velor upholstery',
 'Miesiac_publikacji_oferty',
 'minivan',
 'AUX socket',
 '4x4 (attached manually)',
 'Rear parking sensors',
 'Hook',
 'Immobilizer',
 'coupe',
 'Used',
 'Rok_startu_produkcji',
 'Typ_silnika',
 'Przebieg_km',
 'Rear view camera',
 'Front wheels',
 'Heated rear seats',
 'compact',
 'Gasoline + LPG',
 'Electric rear windows',
 'Electrically adjustable seats',
 'Fog lights',
 'Twilight sensor',
 'Lane assistant',
 'ABS',
 'Kierownica_strona',
 'TV tuner',
 'Airbag protecting

In [228]:
dtype_dict = {
    # Category columns
    "Naped": "category",
    "Skrzynia_biegow": "category",
    "Typ_nadwozia": "category",
    "Kolor": "category",
    "Kraj_pochodzenia": "category",

    # String columns
    "Marka_pojazdu": "category",
    "Model_pojazdu": "string",
    # "Wersja_pojazdu": "string",
    # "Nazwa_generacji": "string",
    "Typ_silnika": "category",

    # Float columns
    "Rok_produkcji": "float64",
    "Przebieg_km": "float64",
    "Moc_KM": "float64",
    "Pojemnosc_cm3": "float64",
    "Emisja_CO2": "float64",
    "Liczba_drzwi": "float64",
    "Rok_startu_produkcji": "float64",
    "Rok_konca_produkcji": "float64",
    'Wspolczynnik_Rok_produkcji': "float64",
    'Wspolczynnik_Przebieg_km': "float64",
    'Wspolczynnik_Moc_KM': "float64",
    'Wspolczynnik_Pojemnosc_cm3': "float64",
    'Wspolczynnik_Liczba_drzwi': "float64",
    'Wspolczynnik_Automatic': "float64",
    'Wspolczynnik_Manual': "float64",
    'Wspolczynnik_SUV': "float64",
    'Wspolczynnik_sedan': "float64",
    'Wspolczynnik_station_wagon': "float64",

    # Integer columns
    "Rok_publikacji_oferty": "float",
    "Miesiac_publikacji_oferty": "float",

    # Boolean columns
    "Xenon lights": "float",
    "Factory radio": "float",
    "Velor upholstery": "float",
    "DVD player": "float",
    "Start-Stop system": "float",
    "Electrically adjustable seats": "float",
    "Fog lights": "float",
    "Shift paddles": "float",
    "Passengers airbag": "float",
    "Daytime running lights": "float",
    "Power steering": "float",
    "Four-zone air conditioning": "float",
    "Sunroof": "float",
    "GPS navigation": "float",
    "Manual air conditioning": "float",
    "Heated side mirrors": "float",
    "Parking assistant": "float",
    "ESP(stabilization of the track)": "float",
    "Immobilizer": "float",
    "Leather upholstery": "float",
    "Lane assistant": "float",
    "Drivers airbag": "float",
    "USB socket": "float",
    "Electric front windows": "float",
    "Blind spot sensor": "float",
    "SD socket": "float",
    "Speed limiter": "float",
    "Electrochromic side mirrors": "float",
    "LED lights": "float",
    "AUX socket": "float",
    "Auxiliary heating": "float",
    "Heated front seats": "float",
    "Rain sensor": "float",
    "Rear parking sensors": "float",
    "Twilight sensor": "float",
    "Front side airbags": "float",
    "Automatic air conditioning": "float",
    "Roof rails": "float",
    "Central locking": "float",
    "ABS": "float",
    "CD": "float",
    "Bluetooth": "float",
    "Dual zone air conditioning": "float",
    "Alloy wheels": "float",
    "Cruise control": "float",
    "Airbag protecting the knees": "float",
    "TV tuner": "float",
    "Rear side airbags": "float",
    "Multifunction steering wheel": "float",
    "Air curtains": "float",
    "On-board computer": "float",
    "Active cruise control": "float",
    "CD changer": "float",
    "Electrochromic rear view mirror": "float",
    "Tfloated windows": "float",
    "Adjustable suspension": "float",
    "Rear view camera": "float",
    "MP3": "float",
    "Heated rear seats": "float",
    "Electric rear windows": "float",
    "Alarm": "float",
    "Hook": "float",
    "Heated windscreen": "float",
    "Isofix": "float",
    "Front parking sensors": "float",
    "Panoramic roof": "float",
    "Aftermarket radio": "float",
    "HUD(head-up display)": "float",
    "Electrically adjustable mirrors": "float",
    "ASR (traction control)": "float",
    'Marka_pojazdu_encoded': "float",
     'Marka_Audi': "float",
     'Marka_BMW': "float",
     'Marka_Citroën': "float",
     'Marka_Fiat': "float",
     'Marka_Ford': "float",
     'Marka_Honda': "float",
     'Marka_Hyundai': "float",
     'Marka_Jeep': "float",
     'Marka_Mazda': "float",
     'Marka_Mercedes-Benz': "float",
     'Marka_Mitsubishi': "float",
     'Marka_Nissan': "float",
     'Marka_Opel': "float",
     'Marka_Peugeot': "float",
     'Marka_Renault': "float",
     'Marka_Seat': "float",
     'Marka_Suzuki': "float",
     'Marka_Toyota': "float",
     'Marka_Volkswagen': "float",
     'Marka_Volvo': "float",
     'Marka_Škoda': "float",

    # Object columns (to be handled separately)
    "Pierwszy_wlasciciel": "float",  # Leave as object or map to category if needed
    "Cena": "float",  # Leave as object or convert to float after cleaning
    "Rok_pierwszej_rejestracji": "float",  # Leave as object or convert to datetime
    "Miesiac_pierwszej_rejestracji": "float",  # Leave as object or convert to datetime
    "Wojewodztwo": "category",  # Leave as object or map to category
    # "Miejscowosc": "object",  # Leave as object or map to category
}
ls = list(set(list(dtype_dict.keys())) & set(common_cols))
new_dtype_dict = {}
for element in ls:
    new_dtype_dict[element] = dtype_dict[element]

df_train = df_train[ls].astype(new_dtype_dict)
df_test = df_test[ls].astype(new_dtype_dict)

In [229]:
# Uzupełnianie nulli w numerycznych danych
for col in df_train.columns:
    col_type = df_train[col].dtype
    if pd.api.types.is_integer_dtype(col_type) and pd.api.types.is_float_dtype(col_type):
        df_train.loc[:, col_type] = df_train[col_type].fillna(0)
        df_train.loc[:, col_type] = df_train[col_type].astype(int)
        df_test.loc[:, col_type] = df_test.loc[:, col_type].fillna(0)
        df_test.loc[:, col_type] = df_test[col_type].astype(int)


In [230]:
# df_train["Marka_Model"] = (df_train["Marka_pojazdu"].astype("str") + " " + df_train["Model_pojazdu"].astype("str")).astype("category")
#
# df_test["Marka_Model"] = (df_test["Marka_pojazdu"].astype("str") + " " + df_test["Marka_pojazdu"].astype("str")).astype("category")

In [231]:
non_number_columns = df_train.select_dtypes(exclude=['float', "int"]).columns
df_train.select_dtypes(exclude=['float', "int"])

,Kraj_pochodzenia,Typ_silnika,Wojewodztwo,Marka_pojazdu,Model_pojazdu
ID,,,,,
1,NaN,NaN,ŁÓDZKIE,Renault,Grand Espace
2,NaN,1.6 16V,ŚLĄSKIE,Renault,Megane
3,Denmark,1.6 CDTI,NaN,Opel,Zafira
4,NaN,1.6 TDCi,NaN,Ford,Focus
5,Poland,1.8,NaN,Toyota,Avensis
...,...,...,...,...,...
135393,Poland,NaN,DOLNOŚLĄSKIE,Opel,Astra
135394,NaN,NaN,MAZOWIECKIE,Mercedes-Benz,Vito
135395,NaN,NaN,MAZOWIECKIE,NaN,Zafira


In [232]:
from category_encoders import TargetEncoder

encoders_dict = {}
refactored_columns = []
for column in non_number_columns:
    # Inicjalizacja kodera
    encoders_dict[column] = TargetEncoder()

    # Kodowanie danych treningowych
    df_train[f"{column}_refactored"] = encoders_dict[column].fit_transform(
        df_train[column],
        df_train["Cena"]  # Target
    ).astype(float)

    # Kodowanie danych testowych
    df_test[f"{column}_refactored"] = encoders_dict[column].transform(
        df_test[column]
    ).astype(float)

    refactored_columns.append(f"{column}_refactored")




# Sprawdź typy danych
print(df_train[refactored_columns])


        Kraj_pochodzenia_refactored  Typ_silnika_refactored  \
ID                                                            
1                      58334.682371            75984.062286   
2                      58334.682371            16233.292419   
3                      34549.597590            42221.292804   
4                      58334.682371            19234.850799   
5                      97951.590117            18626.524549   
...                             ...                     ...   
135393                 97951.590117            75984.062286   
135394                 58334.682371            75984.062286   
135395                 58334.682371            75984.062286   
135396                 34032.482917            52513.695382   
135397                 97951.590117            75984.062286   

        Wojewodztwo_refactored  Marka_pojazdu_refactored  \
ID                                                         
1                 80521.465627              35345.305911   


In [233]:
df_train.drop(columns=non_number_columns, inplace=True)
df_test.drop(columns=non_number_columns, inplace=True)

In [234]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 135397 entries, 1 to 135397
Data columns (total 83 columns):
 #   Column                           Non-Null Count   Dtype  
---  ------                           --------------   -----  
 0   Alloy wheels                     132144 non-null  float64
 1   Rok_produkcji                    125572 non-null  float64
 2   Panoramic roof                   132144 non-null  float64
 3   On-board computer                132144 non-null  float64
 4   Central locking                  132144 non-null  float64
 5   Xenon lights                     132144 non-null  float64
 6   Sunroof                          132144 non-null  float64
 7   Active cruise control            132144 non-null  float64
 8   Rear side airbags                132144 non-null  float64
 9   CD changer                       132144 non-null  float64
 10  Shift paddles                    132144 non-null  float64
 11  Auxiliary heating                132144 non-null  float64
 12  ASR (tr

In [235]:
from sklearn.impute import SimpleImputer

# Krok 1: Imputacja na danych treningowych
imputer = SimpleImputer(strategy='median')
X_train_imputed = imputer.fit_transform(df_train.drop(columns=['Cena']))
y_train = df_train['Cena'].values

# Krok 2: Transformacja danych testowych (BEZ fit_transform!)
X_test_imputed = imputer.transform(df_test.drop(columns=['Cena']))
y_test = df_test['Cena'].values

In [236]:
# Downcasting data types
def reduce_dtypes(df: pd.DataFrame) -> pd.DataFrame:
    for col in df.columns:
        col_type = df[col].dtype
        if pd.api.types.is_numeric_dtype(col_type):
            if pd.api.types.is_integer_dtype(col_type):
                df[col] = pd.to_numeric(df[col], downcast='integer')
            elif pd.api.types.is_float_dtype(col_type):
                df[col] = pd.to_numeric(df[col], downcast='float')
    return df

In [237]:
df_train = reduce_dtypes(df_train)
df_test = reduce_dtypes(df_test)

In [238]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 135397 entries, 1 to 135397
Data columns (total 83 columns):
 #   Column                           Non-Null Count   Dtype  
---  ------                           --------------   -----  
 0   Alloy wheels                     132144 non-null  float32
 1   Rok_produkcji                    125572 non-null  float32
 2   Panoramic roof                   132144 non-null  float32
 3   On-board computer                132144 non-null  float32
 4   Central locking                  132144 non-null  float32
 5   Xenon lights                     132144 non-null  float32
 6   Sunroof                          132144 non-null  float32
 7   Active cruise control            132144 non-null  float32
 8   Rear side airbags                132144 non-null  float32
 9   CD changer                       132144 non-null  float32
 10  Shift paddles                    132144 non-null  float32
 11  Auxiliary heating                132144 non-null  float32
 12  ASR (tr

## Szukanie najlepszych parametrów

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBRegressor
import numpy as np
import cupy as cp

X_train = df_train.loc[:, df_train.columns != "Cena"].to_numpy()
y_train = df_train["Cena"].to_numpy()
X_test = df_test.loc[:, df_test.columns != "Cena"].to_numpy()

# Parametry do tuningu
param_grid_optimized = {
    "n_estimators": [500, 700, 1000],       # Zwiększona liczba drzew dla lepszej generalizacji
    "max_depth": [6, 8, 10],                # Płytsze drzewa (wiele cech binarnych)
    "learning_rate": [0.01, 0.05, 0.1],     # Niższe learning rate dla stabilności
    "subsample": [0.8, 0.9, 1.0],           # Losowa próbka danych dla każdego drzewa
    "colsample_bytree": [0.6, 0.7, 0.8],    # 60-80% cech na drzewo (wiele zmiennych)
    "gamma": [0, 0.1, 0.2],                 # Kontrola podziałów węzłów
    "reg_alpha": [0, 0.1, 1, 10],           # L1 - dobre dla danych rzadkich
    "reg_lambda": [0, 0.1, 1, 10],          # L2 - stabilizacja wag
    "min_child_weight": [1, 5, 10]          # Dodatkowa kontrola przeuczenia
}
param_grid_restricted = {
    # 21 min
    "n_estimators": [ 500, 600],  # Zamiast [50, 100, 200, 300]
    "max_depth": [17],  # Zamiast [3, 5, 7, 9]
    "learning_rate": [0.04, 0.05],  # Zamiast [0.01, 0.05, 0.1, 0.2]
    "subsample": [0,9, 1],  # Zamiast [0.6, 0.8, 1.0]
    "colsample_bytree": [0.6],  # Zamiast [0.6, 0.8, 1.0]
    "gamma": [0.3, 0.04],  # Zamiast [0, 0.1, 0.2]
    "reg_alpha": [0.8, 1],  # Zamiast [0, 0.1, 1]
    "reg_lambda": [0.1, 0.06],  # Zamiast [0, 0.1, 1]
}

# Parametry specyficzne dla GPU
params = {
    "device" : "cuda",
    "tree_method" : "hist",
    "objective": "reg:squarederror",  # Zadanie regresji
    "eval_metric": "rmse",  # Metryka RMSE
    # "gpu_id": 0,  # Użyj GPU o indeksie 0
    # "predictor": "gpu_predictor",  # Użyj GPU do predykcji
}

# Inicjalizacja modelu XGBRegressor z parametrami GPU
xgb = XGBRegressor(**params, enable_categorical=True, random_state=42)

# Konfiguracja Grid Search
# grid_search = GridSearchCV(
#     estimator=xgb,
#     param_grid=param_grid_restricted,
#     scoring="neg_mean_squared_error",  # Metryka do optymalizacji (RMSE)
#     cv=5,  # 5-krotna walidacja krzyżowa
#     n_jobs=-1,  # Użyj wszystkich dostępnych rdzeni CPU
#     verbose=2,  # Wyświetl postęp
# )

random_search = RandomizedSearchCV(
    estimator=xgb,
    param_distributions=param_grid_optimized,
    n_iter=160,  # Przetestuj tylko 50 losowych kombinacji
    scoring="neg_mean_squared_error",
    cv=3,
    n_jobs=1,
    verbose=3,
    random_state=42,
    error_score='raise'
)

# Trenowanie modelu z Grid Search
# grid_search.fit(X_train, y_train)
random_search.fit(X_train, y_train)

# Najlepsze parametry i wynik
print("Najlepsze parametry:", random_search.best_params_)
print("Najlepszy wynik (Negative MSE):", random_search.best_score_)
print("Najlepszy wynik (RMSE):", np.sqrt(-random_search.best_score_))


# Ocena modelu na zbiorze testowym
best_model = random_search.best_estimator_
y_pred = best_model.predict(X_test)



Fitting 3 folds for each of 160 candidates, totalling 480 fits
[CV 1/3] END colsample_bytree=0.7, gamma=0.1, learning_rate=0.01, max_depth=6, min_child_weight=5, n_estimators=1000, reg_alpha=0, reg_lambda=0.1, subsample=0.8;, score=-1685556517.648 total time=   4.1s
[CV 2/3] END colsample_bytree=0.7, gamma=0.1, learning_rate=0.01, max_depth=6, min_child_weight=5, n_estimators=1000, reg_alpha=0, reg_lambda=0.1, subsample=0.8;, score=-670091863.717 total time=   3.5s
[CV 3/3] END colsample_bytree=0.7, gamma=0.1, learning_rate=0.01, max_depth=6, min_child_weight=5, n_estimators=1000, reg_alpha=0, reg_lambda=0.1, subsample=0.8;, score=-1223033112.218 total time=   3.7s
[CV 1/3] END colsample_bytree=0.6, gamma=0, learning_rate=0.01, max_depth=8, min_child_weight=10, n_estimators=1000, reg_alpha=10, reg_lambda=1, subsample=1.0;, score=-1626656133.096 total time=   5.5s


In [183]:
from xgboost import plot_importance
plot_importance(random_search.best_estimator_)

AttributeError: 'RandomizedSearchCV' object has no attribute 'best_estimator_'

In [208]:
y_pred = best_model.predict(df_test.loc[:, X_train.columns])

In [182]:
param_grid_restricted = {
    'subsample': [1],
    'reg_lambda': [0.06],
    'reg_alpha': [1],
    'n_estimators': [195, 196],
    'max_depth': [11],
    'learning_rate': [0.05],
    'gamma': [0.3],
    'colsample_bytree':[0.6]
}

# Parametry specyficzne dla GPU
params = {
    # "device" : "cuda",
    "tree_method" : "gpu_hist",
    "objective": "reg:squarederror",  # Zadanie regresji
    "eval_metric": "rmse",  # Metryka RMSE
    "gpu_id": 0,  # Użyj GPU o indeksie 0
    "predictor": "gpu_predictor",  # Użyj GPU do predykcji
}

# Inicjalizacja modelu XGBRegressor z parametrami GPU
xgb = XGBRegressor(**params, enable_categorical=True, random_state=42)

random_search = RandomizedSearchCV(
    estimator=xgb,
    param_distributions=param_grid_restricted,
    n_iter=50,  # Przetestuj tylko 50 losowych kombinacji
    scoring="neg_mean_squared_error",
    cv=5,
    n_jobs=-2,
    verbose=3,
    random_state=42
)

# Trenowanie modelu z Grid Search
# grid_search.fit(X_train, y_train)
random_search.fit(df_train.loc[:, df_train.columns != "Cena"],
        df_train["Cena"])

# Najlepsze parametry i wynik
print("Najlepsze parametry:", random_search.best_params_)
print("Najlepszy wynik (Negative MSE):", random_search.best_score_)

# Ocena modelu na zbiorze testowym
best_model = random_search.best_estimator_
y_pred = best_model.predict(df_test[:, (df_test.columns != "Model_pojazdu")])

C:\Users\vipcy\anaconda3\envs\car-prices-prediction\lib\site-packages\sklearn\model_selection\_search.py:317: UserWarning: The total space of parameters 2 is smaller than n_iter=50. Running 2 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Fitting 5 folds for each of 2 candidates, totalling 10 fits


C:\Users\vipcy\anaconda3\envs\car-prices-prediction\lib\site-packages\xgboost\core.py:158: UserWarning: [20:16:29] WARNING: D:\bld\xgboost-split_1742444215012\work\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
C:\Users\vipcy\anaconda3\envs\car-prices-prediction\lib\site-packages\xgboost\core.py:158: UserWarning: [20:16:29] WARNING: D:\bld\xgboost-split_1742444215012\work\src\learner.cc:740: 
Parameters: { "predictor" } are not used.

  warnings.warn(smsg, UserWarning)


Najlepsze parametry: {'subsample': 1, 'reg_lambda': 0.06, 'reg_alpha': 1, 'n_estimators': 196, 'max_depth': 11, 'learning_rate': 0.05, 'gamma': 0.3, 'colsample_bytree': 0.6}
Najlepszy wynik (Negative MSE): -1322341477.6955178


ValueError: feature_names mismatch: ['Sunroof', 'Electrochromic side mirrors', 'Shift paddles', 'Heated front seats', 'CD', 'Rain sensor', 'Parking assistant', 'Alloy wheels', 'Pierwszy_wlasciciel', 'Electrically adjustable seats', 'Heated side mirrors', 'TV tuner', 'Four-zone air conditioning', 'ASR (traction control)', 'Rok_publikacji_oferty', 'HUD(head-up display)', 'CD changer', 'Power steering', 'Wojewodztwo', 'Immobilizer', 'Fog lights', 'ESP(stabilization of the track)', 'On-board computer', 'Front side airbags', 'Heated rear seats', 'Rok_startu_produkcji', 'Bluetooth', 'Isofix', 'Leather upholstery', 'Rear side airbags', 'ABS', 'Passengers airbag', 'SD socket', 'LED lights', 'Kraj_pochodzenia', 'Central locking', 'Rok_pierwszej_rejestracji', 'Electrically adjustable mirrors', 'MP3', 'Air curtains', 'Miesiac_publikacji_oferty', 'Kolor', 'Airbag protecting the knees', 'Daytime running lights', 'Pojemnosc_cm3', 'Typ_silnika', 'Manual air conditioning', 'Liczba_drzwi', 'Panoramic r

In [211]:
print(df_test.shape)
print(y_pred.shape)

anwser_df = pd.DataFrame({"ID": df_test.index,"Cena": y_pred})
anwser_df.to_csv("Anwser.csv", index=False)

(72907, 92)
(72907,)


In [73]:
# Parametry specyficzne dla GPU
param_grid_restricted = {
    # 21 min
    "n_estimators": [200],  # Zamiast [50, 100, 200, 300]
    "max_depth": [7],  # Zamiast [3, 5, 7, 9]
    "learning_rate": [0.05],  # Zamiast [0.01, 0.05, 0.1, 0.2]
    "subsample": [1],  # Zamiast [0.6, 0.8, 1.0]
    "colsample_bytree": [0.8],  # Zamiast [0.6, 0.8, 1.0]
    "gamma": [0],  # Zamiast [0, 0.1, 0.2]
    "reg_alpha": [0.1],  # Zamiast [0, 0.1, 1]
    "reg_lambda": [0.1],  # Zamiast [0, 0.1, 1]
}
params = {
    "tree_method": "gpu_hist",  # Użyj GPU do budowy drzew
    "objective": "reg:squarederror",  # Zadanie regresji
    "eval_metric": "rmse",  # Metryka RMSE
    "gpu_id": 0,  # Użyj GPU o indeksie 0
    "predictor": "gpu_predictor",  # Użyj GPU do predykcji
}

# Inicjalizacja modelu XGBRegressor z parametrami GPU
xgb = XGBRegressor(**params, enable_categorical=True, random_state=42)

# Konfiguracja Grid Search
grid_search = GridSearchCV(
    estimator=xgb,
    param_grid=param_grid_restricted,
    scoring="neg_mean_squared_error",  # Metryka do optymalizacji (RMSE)
    cv=5,  # 5-krotna walidacja krzyżowa
    n_jobs=-1,  # Użyj wszystkich dostępnych rdzeni CPU
    verbose=2,  # Wyświetl postęp
)

# random_search = RandomizedSearchCV(
#     estimator=xgb,
#     param_distributions=param_grid,
#     n_iter=50,  # Przetestuj tylko 50 losowych kombinacji
#     scoring="neg_mean_squared_error",
#     cv=5,
#     n_jobs=-1,
#     verbose=2,
#     random_state=42,
# )

# Trenowanie modelu z Grid Search
grid_search.fit(X_train, y_train)

# Najlepsze parametry i wynik
print("Najlepsze parametry:", grid_search.best_params_)
print("Najlepszy wynik (Negative MSE):", grid_search.best_score_)

# Ocena modelu na zbiorze testowym
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

from sklearn.metrics import mean_squared_error, r2_score

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print(f"RMSE na zbiorze testowym: {rmse}")
print(f"R² na zbiorze testowym: {r2}")

Fitting 5 folds for each of 1 candidates, totalling 5 fits


C:\Users\vipcy\anaconda3\envs\car-prices-prediction\lib\site-packages\xgboost\core.py:158: UserWarning: [10:57:44] WARNING: D:\bld\xgboost-split_1742444215012\work\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
C:\Users\vipcy\anaconda3\envs\car-prices-prediction\lib\site-packages\xgboost\core.py:158: UserWarning: [10:57:44] WARNING: D:\bld\xgboost-split_1742444215012\work\src\learner.cc:740: 
Parameters: { "predictor" } are not used.

  warnings.warn(smsg, UserWarning)


Najlepsze parametry: {'colsample_bytree': 0.8, 'gamma': 0, 'learning_rate': 0.05, 'max_depth': 7, 'n_estimators': 200, 'reg_alpha': 0.1, 'reg_lambda': 0.1, 'subsample': 1}
Najlepszy wynik (Negative MSE): -1444583433.5396085
RMSE na zbiorze testowym: 38406.08798380288
R² na zbiorze testowym: 0.7980510874610559


C:\Users\vipcy\anaconda3\envs\car-prices-prediction\lib\site-packages\xgboost\core.py:158: UserWarning: [10:57:45] WARNING: D:\bld\xgboost-split_1742444215012\work\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


In [174]:
final_anwser = best_model.predict(df_test)

IndexError: Boolean index has wrong length: 89 instead of 27080

In [129]:
anwser_df = pd.DataFrame()
anwser_df.loc[:, "ID"] = X_test.index
anwser_df.loc[:, "Cena"] = final_anwser

In [130]:
anwser_df

,ID,Cena
0,22440,38665.882812
1,4159,20987.648438
2,80228,3643.710938
3,89706,7591.453125
4,74416,9124.679688
...,...,...
27075,124795,19188.574219
27076,64019,28267.101562
27077,107470,44034.601562
27078,47227,20445.632812


In [26]:
df_train["Marka_pojazdu"].unique()

['Renault', 'Opel', 'Ford', 'Toyota', 'BMW', ..., 'Autobianchi', 'Nysa', 'Vanderhall', 'Zaporożec', 'Saturn']
Length: 103
Categories (102, object): ['Abarth', 'Acura', 'Aixam', 'Alfa Romeo', ..., 'Zaporożec', 'Zastava', 'Škoda', 'Żuk']